# 1. Data Cleaning & Preperation

#### Load the data

In [9]:
from statistics import correlation

import pandas as pd
ad_data=pd.read_csv('ad_data.csv')
print(ad_data.columns)
# print(ad_data.head())

Index(['Unnamed: 0', 'timestamp', 'page_views', 'ad_spend', 'is_weekend',
       'day_of_week', 'is_holiday'],
      dtype='object')


#### Drop the Unnamed: 0 column

In [10]:
ad_data=ad_data.drop(columns=['Unnamed: 0'])
print(ad_data.head())


    timestamp   page_views    ad_spend  is_weekend day_of_week  is_holiday
0  2022-01-01  1165.962767  194.901425        True    Saturday        True
1  2022-01-02  1191.485835  175.852071        True      Sunday       False
2  2022-01-03  1266.928269  269.430656       False      Monday       False
3  2022-01-04  1364.193858  295.690896       False     Tuesday       False
4  2022-01-05  1013.282520  242.975399       False   Wednesday       False


#### Convert timestamp to datetime format

In [11]:
ad_data['timestamp']=pd.to_datetime(ad_data['timestamp'])
print("Timestamp is converted to datetime")

Timestamp is converted to datetime


#### Set timestamp as the DataFrame index

In [12]:
ad_data.set_index('timestamp', inplace=True)
print("Timestamp is converted to datetime")

Timestamp is converted to datetime


#### Check for missing values and handle them

In [13]:
print(ad_data.isnull().sum())

page_views     0
ad_spend       0
is_weekend     0
day_of_week    0
is_holiday     0
dtype: int64


#### Check for and remove any duplicate rows

In [14]:
print(ad_data.duplicated().sum())

0


#### Check data types and ensure all are correct

In [18]:
print(ad_data.dtypes)

page_views     float64
ad_spend       float64
is_weekend        bool
day_of_week     object
is_holiday        bool
dtype: object


# 2. Exploratory Data Analysis (EDA)

#### Find basic stats (mean, median, std) for page_views and ad_spend

##### Find the mean

In [22]:
#mean is the average value
page_views_mean=ad_data['page_views'].mean()
ad_spend=ad_data['ad_spend'].mean()
print("Page Views mean: ",page_views_mean)
print("Ad Spend mean: ",ad_spend)

Page Views mean:  1553.196103873606
Ad Spend mean:  249.5184663423933


#### Find the median

In [23]:
# median is the middle value when data is sorted
page_views_median=ad_data['page_views'].median()
ad_spend_median=ad_data['ad_spend'].median()
print("Page Views median: ",page_views_median)
print("Ad Spend median: ",ad_spend_median)

Page Views median:  1562.0695948974926
Ad Spend median:  240.46575170748


#### Find the standard deviation

In [24]:
page_views_Std=ad_data['page_views'].std()
ad_spend_Std=ad_data['ad_spend'].std()
print("Page Views standard deviation: ",page_views_Std)
print("Ad Spend standard deviation: ",ad_spend_Std)

Page Views standard deviation:  277.14416046980796
Ad Spend standard deviation:  73.05512902240615


####  average page views and ad spend by day_of_week

In [35]:
adSpend_weekdays=ad_data.loc[ad_data['is_weekend']==False,'ad_spend'].mean()
pageView_weekdays=ad_data.loc[ad_data['is_weekend']==False,'page_views'].mean()
print("Average ad spend on weekdays:",round(adSpend_weekdays,2))
print("Average page view on weekdays:",round(pageView_weekdays,2))

Average ad spend on weekdays: 268.66
Average page view on weekdays: 1519.59


####  Compare performance on weekend vs weekday

In [36]:
comparison = (
    ad_data.groupby('is_weekend')[['ad_spend', 'page_views']].mean().rename(index={False: 'Weekday', True: 'Weekend'}).round(2)
)
print(comparison)


            ad_spend  page_views
is_weekend                      
Weekday       268.66     1519.59
Weekend       201.86     1636.89


#### Analyze the effect of is_holiday on page_views

In [38]:
compare_holidays=ad_data.groupby('is_holiday')[['page_views']].mean().rename(index={False:'Non Holiday', True: 'Holiday'}).round(2)
print(compare_holidays)

             page_views
is_holiday             
Non Holiday     1552.57
Holiday         1610.17


#### Find correlation between ad_spend and page_views

In [40]:
# Correlation means how much two things are connected i.e. directly proportional or inversely proportional
corelation=ad_data['ad_spend'].corr(ad_data['page_views'])
print("Correlation between Ad Spend & Page views: ",round(correlation,2))

Correlation between Ad Spend & Page views:  -0.0


# 3. Metrics & Aggregation

#### Create a new column: views_per_dollar = page_views / ad_spend

In [42]:
ad_data['view_per_dollar']=ad_data['ad_spend']/ad_data['page_views']
print(ad_data.head(5))

             page_views    ad_spend  is_weekend day_of_week  is_holiday  \
timestamp                                                                 
2022-01-01  1165.962767  194.901425        True    Saturday        True   
2022-01-02  1191.485835  175.852071        True      Sunday       False   
2022-01-03  1266.928269  269.430656       False      Monday       False   
2022-01-04  1364.193858  295.690896       False     Tuesday       False   
2022-01-05  1013.282520  242.975399       False   Wednesday       False   

            view_per_dollar  
timestamp                    
2022-01-01         0.167159  
2022-01-02         0.147591  
2022-01-03         0.212664  
2022-01-04         0.216751  
2022-01-05         0.239790  


#### Group data by day_of_week and calculate average views_per_dollar

In [45]:
view_perDollar_weekdays=ad_data.loc[ad_data['is_weekend']==False,'view_per_dollar'].mean()
print("Average Views per dollar: ", round(view_perDollar_weekdays,2))

Average Views per dollar:  0.18


#### Resample data by week and month for trend analysis

In [64]:
weekly_data=ad_data.resample('W')[['ad_spend', 'page_views']].sum()
print("Weekly Data:\n ",round(weekly_data.head(4),8))
monthly_data=ad_data.resample('ME')[['ad_spend', 'page_views']].sum()
print("Monthly Data:\n ",round(monthly_data.head(4),8))

Weekly Data:
                 ad_spend   page_views
timestamp                           
2022-01-02   370.753496  2357.448603
2022-01-09  1717.388037  8282.401647
2022-01-16  1479.646678  8702.449200
2022-01-23  1558.654673  8115.337922
Monthly Data:
                 ad_spend    page_views
timestamp                            
2022-01-31  6862.616595  37655.674858
2022-02-28  6319.738621  34350.350564
2022-03-31  7230.862649  39531.092265
2022-04-30  6851.773536  39290.942654


#### Apply 7-day and 30-day rolling averages to page_views

In [73]:
weekly_average_views=ad_data.resample('W')[['page_views']].mean()
print("Weekly Average Views:\n ",round(weekly_average_views,2).head(5))
monthly_average_views=ad_data.resample('ME')[['page_views']].mean()
print("Average Monthly Views:\n ",round(monthly_average_views,2).head(5))

Weekly Average Views:
              page_views
timestamp             
2022-01-02     1178.72
2022-01-09     1183.20
2022-01-16     1243.21
2022-01-23     1159.33
2022-01-30     1239.98
Average Monthly Views:
              page_views
timestamp             
2022-01-31     1214.70
2022-02-28     1226.80
2022-03-31     1275.20
2022-04-30     1309.70
2022-05-31     1367.82


# 4.  Insights & Analysis

#### Identify top 10 days with highest views_per_dollar

In [84]:
top10_views_per_dollar=ad_data.sort_values('view_per_dollar',ascending=False)
print(top10_views_per_dollar['view_per_dollar'].head(10))

timestamp
2022-09-01    0.561147
2022-05-06    0.558999
2022-08-31    0.547987
2022-09-02    0.435255
2022-05-12    0.430514
2023-05-10    0.429679
2023-05-11    0.402647
2022-11-10    0.400442
2022-08-30    0.399939
2024-11-28    0.378644
Name: view_per_dollar, dtype: float64


#### Determine if ad spend is higher or lower during weekends

In [87]:
compare_adSpend=ad_data.groupby('is_weekend')[['ad_spend']].mean().rename(index={False:'Weekdays',True:'Weekend'})
print(round(compare_adSpend,2))

            ad_spend
is_weekend          
Weekdays      268.66
Weekend       201.86


# 5. Final Summary

- Average spending is lower on weekends(201.86) as compared to weekdays(268.66). Weekend page views are higher than weekdays
- Average page views are high on holidays(1610.17) than holidays(1552.57)
- The average per dollar is 0.18

#### Save cleaned data to a new CSV file

In [88]:
ad_data.to_csv('processed_adData.csv')
print("Data saved to new file: Processed ad data")

Data saved to new file: Processed ad data
